In [1]:
import polars
import hvplot.polars

import holoviews as hv
import utils
import indicatory.names as names

from polars import col

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [2]:
elements = 100
amd = utils.load_asset_data("AMD").tail(elements)
amd.head()

Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Time Zone
date,f64,f64,f64,f64,i64,f64,f64,str
2024-03-18,193.289993,193.5,188.050003,190.649994,59306900,0.0,0.0,"""EDT"""
2024-03-19,183.160004,185.610001,177.360001,181.419998,83509500,0.0,0.0,"""EDT"""
2024-03-20,182.029999,182.679993,174.0,179.729996,81773100,0.0,0.0,"""EDT"""
2024-03-21,185.970001,187.690002,177.660004,178.679993,75265600,0.0,0.0,"""EDT"""
2024-03-22,177.210007,180.759995,175.050003,179.649994,57832100,0.0,0.0,"""EDT"""


In [3]:
import indicatory.oscillators as osc

In [11]:
short = 5
long = 20
ppo = osc.percentage_price_oscillator(amd, short_window_size=short, long_window_size=long)
simple_rsi = osc.simple_relative_strength_index(amd)
rsi = osc.relative_strength_index(amd)
so = osc.stochastic_oscillator(amd)
dpo = osc.detrend_price_oscillator(amd)

In [5]:
periods = elements - long
ohlc = utils.ohlc_plot(amd.tail(periods))
ppo_plot = utils.scatter_plot(ppo.tail(periods), value_fields=[names.ppo(column_name="Close", short_window_size=short, long_window_size=long)])
hline = hv.HLine(0).options(color='black', line_width=1)

hv.Layout(ohlc + ppo_plot * hline).cols(1) 

:Layout
   .Overlay.I  :Overlay
      .Segments.I   :Segments   [Date,Low,Date,High]   (Open,High,Low,Close,lbound,ubound)
      .Rectangles.I :Rectangles   [lbound,Open,ubound,Close]   (Open,High,Low,Close,lbound,ubound)
   .Overlay.II :Overlay
      .Scatter.I :Scatter   [Date]   (PPO 5/20 Close)
      .HLine.I   :HLine   [x,y]

In [6]:
overbought_threshold = hv.HLine(70).options(line_width=1, color="red")
oversold_threshold = hv.HLine(30).opts(line_width=1, color="red")
simple_rsi_plot = utils.scatter_plot(simple_rsi.tail(periods), value_fields=[names.rsi(10)])
rsi_plot = utils.scatter_plot(rsi.tail(periods), value_fields=[names.rsi(10)], color="darkblue")

hv.Layout(ohlc + simple_rsi_plot * overbought_threshold * oversold_threshold + rsi_plot * overbought_threshold * oversold_threshold).cols(1)

:Layout
   .Overlay.I   :Overlay
      .Segments.I   :Segments   [Date,Low,Date,High]   (Open,High,Low,Close,lbound,ubound)
      .Rectangles.I :Rectangles   [lbound,Open,ubound,Close]   (Open,High,Low,Close,lbound,ubound)
   .Overlay.II  :Overlay
      .Scatter.I :Scatter   [Date]   (RSI 10)
      .HLine.I   :HLine   [x,y]
      .HLine.II  :HLine   [x,y]
   .Overlay.III :Overlay
      .Scatter.I :Scatter   [Date]   (RSI 10)
      .HLine.I   :HLine   [x,y]
      .HLine.II  :HLine   [x,y]

In [10]:
so_fast_plot = utils.step_plot(so.tail(periods), value_fields=[names.fast_k(5), names.fast_d(5)], color=["green", "orange"])
so_slow_plot = utils.step_plot(so.tail(periods), value_fields=[names.slow_k(5), names.slow_d(5)], color=["blue", "purple"])

hv.Layout(ohlc + so_fast_plot * overbought_threshold * oversold_threshold + so_slow_plot * overbought_threshold * oversold_threshold).cols(1)

:Layout
   .Overlay.I   :Overlay
      .Segments.I   :Segments   [Date,Low,Date,High]   (Open,High,Low,Close,lbound,ubound)
      .Rectangles.I :Rectangles   [lbound,Open,ubound,Close]   (Open,High,Low,Close,lbound,ubound)
   .Overlay.II  :Overlay
      .NdOverlay.I :NdOverlay   [Variable]
         :Curve   [Date]   (value)
      .HLine.I     :HLine   [x,y]
      .HLine.II    :HLine   [x,y]
   .Overlay.III :Overlay
      .NdOverlay.I :NdOverlay   [Variable]
         :Curve   [Date]   (value)
      .HLine.I     :HLine   [x,y]
      .HLine.II    :HLine   [x,y]

In [14]:
dpo_plot = utils.bar_plot(dpo.tail(periods), value_field=names.dpo(column_name="Close", window_size=10))

hv.Layout(ohlc + dpo_plot).cols(1)

:Layout
   .Overlay.I :Overlay
      .Segments.I   :Segments   [Date,Low,Date,High]   (Open,High,Low,Close,lbound,ubound)
      .Rectangles.I :Rectangles   [lbound,Open,ubound,Close]   (Open,High,Low,Close,lbound,ubound)
   .Bars.I    :Bars   [Date]   (DPO 10 Close)